In [ ]:
import copy
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

We will go through an example of a classification task.

**Goals**:
- set up a binary classifer using scikit-learn
- classify cells from the rd10 dataset as either healthy (WT genotype) or diseased (rd10 homozygous genotype)
- evaluate the model's performance using a test dataset
- choose a penalty (options: None, "l1", "l2" or "elasticnet") and a suitable solver


Scikit-learn documentation:
- [Logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

# 1. Load dataset


We will use the same dataset from the previous tutorial
- Download dataset from the course google drive folder: `IOB (General) > PhD Program > PC Spring 2025 > Data Analysis course June 2025 > session3 > practical > adata_course_filt.h5ad`

In [ ]:
# NOTE: Assumes that the anndata object is stored in a directory named "data" located one
#       level up from the current directory
datadir = Path("..", "data")

adata = sc.read_h5ad(Path(datadir, "adata_course_filt.h5ad"))
adata

Let's have a look at the labels that we are going to use in this tutorial:

In [ ]:
adata.obs["genotype"].value_counts()

In [ ]:
# QC gene and cell filters
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Remove multiplets, projecting_neurons and undefined cells
mask = adata.obs["cell_type"].str.contains("mult|undefined|neuron")
adata = adata[~mask].copy()

# Let's use highly_variable_genes to rank genes
sc.pp.highly_variable_genes(adata, n_top_genes=10000, flavor="seurat_v3")
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# 2. Baseline model

**Task**: Use the pre-selected features (i.e., highly-variable genes) to train a classifier and report the accuracy of the predictions on the test set


Note that the number of genes is greater than the number of cells. Let's start by selecting the top 200 highly variable genes

In [ ]:
adata_hv = adata[:, adata.var["highly_variable_rank"] < 200]
features = adata_hv.X
target = adata_hv.obs["genotype"]
features.shape, target.shape

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Trick to get metadata associated with partitions
# df_train, df_test, y_train_, y_test_ = train_test_split(adata_hv.obs, target, test_size=0.3, random_state=42)

Let's check that the class labels are well-balance in both the training and test sets

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
# Define the logistic regression model
logreg = LogisticRegression(penalty=None, max_iter=1000)

# Train the logistic regression model
logreg.fit(X_train, y_train)

In [ ]:
print(f"Accuracy of logistic regression classifier on training set: {logreg.score(X_train, y_train):.4f}")
print(f"Accuracy of logistic regression classifier on test set: {logreg.score(X_test, y_test):.4f}")

In [ ]:
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

**Question**: Which of the two labels (`hom` or `wt`) can we predict more accurately?

The `wt` label is predicted more accurately overall due to its higher recall and F1-score, despite the slightly lower precision compared to the `hom` label.
In this context, precision refers to the proportion of correctly labeled cells among the predicted labels for each class, while recall denotes the proportion of correctly labeled cells among the actual labels for each class. Let's take a look at the confusion matrix: 

In [ ]:
confusion_matrix(y_test, y_pred)

For instance, out of 1979 actual `hom` cells, we correctly identified 1500, yielding a recall of 0.76. Conversely, out of 1803 cells predicted as `hom`, our correct identification of 1500 gives a precision of 0.83.


|            | pred. hom | pred. wt |
|------------|-----------|----------|
| actual hom | 1500      | 479      |
| acutal wt  | 303       | 2022     |



## Feature standardization

Scaling features is generally recommended in machine learning, though not always required. 
Algorithms that rely on gradient descent optimization, such as neural networks, support vector machines, and some linear models (like logistic regression), benefit from scaled features as they can converge faster. Also, algorithms relying on distance measures (such as k-nearest neighbors, k-means) are sensitive to the scale of input features, so it is important to apply feature scaling. Furthermore, when regularization is applied in the loss function, feature scaling ensures coefficients are penalized appropriately.

On the other hand, Tree-based models (such as decision trees, random forests) are generally invariant to the scale of features. Scaling typically does not impact their performance.

In [ ]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(np.asarray(X_train.todense()))
X_scaled_test = scaler.transform(np.asarray(X_test.todense()))

# NOTE: the number of max. iterations is smaller
logreg_scaled = LogisticRegression(penalty=None, max_iter=700)
logreg_scaled.fit(X_scaled_train, y_train)

print(f"Accuracy of logistic regression classifier on training set: {logreg_scaled.score(X_scaled_train, y_train):.4f}")
print(f"Accuracy of logistic regression classifier on test set: {logreg_scaled.score(X_scaled_test, y_test):.4f}")

In [ ]:
idxs = np.where(np.abs(logreg_scaled.coef_[0]) > 15)[0]
plt.scatter(
    logreg.coef_[0],
    logreg_scaled.coef_[0],
    c=adata_hv.var["highly_variable_rank"],
    cmap="viridis",
)
for idx in idxs:
    plt.annotate(
        adata_hv[:, idx].var_names[0],
        (logreg.coef_[0, idx], logreg_scaled.coef_[0, idx]),
        textcoords="offset points",
        xytext=(0,10),
        ha="center"
    )
plt.colorbar()
plt.xlabel("Unstandardized coefficients")
plt.ylabel("Standardized coefficients")
plt.show()

# 3. Perform cross-validation to select regularization strength

Let's now use a larger set of genes. To achieve this, we suggest applying a penalty to constrain the weights/parameters of the model and avoid overfitting.
**Tip:**  Using all genes that pass QC steps would be ideal, but it can take too long to train the model. Therefore, consider a number of genes larger than 200 but fewer than 5000 (depending on your patience!).

We will use grid search to find a suitable regularization strength (`C` as implemented in scikit-learn). Grid search is a fundamental strategy for tuning hyperparameters. The process involves the following steps. For each hyperparameter, define a set of candidate values. Then, train separate models using every possible combination of these values. Finally, select the configuration that yields the lowest validation error.


**Tip:** Select a suitable solver depending on your choice of regularizer

Scikit-learn documentation:
- [Tuning hyper-parameters](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator)

In [ ]:
# Gene subset: extracting the top 1000 highly variable genes.
adata_hv = adata[:, adata.var["highly_variable_rank"] < 1000].copy()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(adata_hv.X, adata_hv.obs["genotype"], test_size=0.3, random_state=42)

scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(np.asarray(X_train.todense()))
X_scaled_test = scaler.transform(np.asarray(X_test.todense()))

## L2 regularization

In [ ]:
# Define the logistic regression model
logreg_l2 = LogisticRegression(penalty="l2", max_iter=500)

# Define the hyperparameter grid for regularization strength (C)
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Perform cross-validation to find the best regularization strength
outfile = Path(datadir, "grid_search_l2.pkl")
if outfile.exists():
    with open(outfile, "rb") as f:
        grid_search_l2 = pickle.load(f)
else:
    grid_search_l2 = GridSearchCV(logreg_l2, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid_search_l2.fit(X_scaled_train, y_train)
    with open(outfile, "wb") as f:
        pickle.dump(grid_search_l2, f)


# Get the best model and the best regularization parameter
best_logreg_l2 = grid_search_l2.best_estimator_
best_C = grid_search_l2.best_params_["C"]

print(f"Best regularization strength (C): {best_C}")

# Train the logistic regression model with the best parameter on the entire training set
best_logreg_l2.fit(X_scaled_train, y_train)

# Make predictions on the test set
y_pred = best_logreg_l2.predict(X_scaled_test)

# Evaluate the model
print(f"Accuracy of training set: {best_logreg_l2.score(X_scaled_train, y_train):.4f}")
print(f"Accuracy on test set: {best_logreg_l2.score(X_scaled_test, y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

results = grid_search_l2.cv_results_
plt.figure(figsize=(5, 3))
plt.plot(param_grid["C"], results["mean_test_score"], marker="o")
plt.xscale("log")
plt.xlabel("Regularization strength (C)")
plt.ylabel("Mean CV Accuracy")
plt.title("Cross-Validation accuracy for different regularization strengths")
plt.grid(True)
plt.show()

In [ ]:
adata_hv.var["coef"] = best_logreg_l2.coef_[0]

mask = adata_hv.var["coef"].abs() > 0.5
plt.scatter(
    adata_hv.var["highly_variable_rank"],
    adata_hv.var["coef"],
)
for idx, row in adata_hv[:, mask].var.iterrows():
    plt.annotate(
        idx,
        (row["highly_variable_rank"], row["coef"]),
        textcoords="offset points",
        xytext=(0,10),
        ha="center",
    )
plt.xlabel("Gene rank")
plt.ylabel("Standardized coefficients")
plt.show()

In this context, we are fitting the probability of being a wild-type cell (`wt`) given its transcriptome. The coefficient for each feature can be exponentiated to get the odds ratio. If the coefficient of a given gene is negative, the odds ratio will be less than 1, indicating a decrease in the odds of the `wt` class as that gene is more expressed. Equivalently,since the logistic function maps the log-odds to probabilities, a negative coefficient implies that as the gene is more expressed, the probability of  being a wild-type cell decreases

## L1 regularization

Some advantages of the "saga" solver:
- Uses a stochastic average gradient descent algorithm
- Supports both L1 and L2 regularization, as well as elastic net regularization (a mix of L1 and L2).
- Suitable for large datasets and high-dimensional data.
- Speed: Efficient for large datasets, especially those with sparse features.

In [ ]:
# Define the logistic regression model
logreg_l1 = LogisticRegression(solver="saga", penalty="l1", max_iter=5000)

# Define the hyperparameter grid for regularization strength (C)
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Perform cross-validation to find the best regularization strength
outfile = Path(datadir, "grid_search_l1.pkl")
if outfile.exists():
    with open(outfile, "rb") as f:
        grid_search_l1 = pickle.load(f)
else:
    grid_search_l1 = GridSearchCV(logreg_l1, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid_search_l1.fit(X_scaled_train, y_train)
    with open(outfile, "wb") as f:
        pickle.dump(grid_search_l1, f)

# Get the best model and the best regularization parameter
best_logreg_l1 = grid_search_l1.best_estimator_
best_C = grid_search_l1.best_params_["C"]

print(f"Best regularization strength (C): {best_C}")

# Train the logistic regression model with the best parameter on the entire training set
best_logreg_l1.fit(X_scaled_train, y_train)

# Make predictions on the test set
y_pred = best_logreg_l1.predict(X_scaled_test)

# Evaluate the model
print(f"Accuracy of training set: {best_logreg_l1.score(X_scaled_train, y_train):.4f}")
print(f"Accuracy on test set: {best_logreg_l1.score(X_scaled_test, y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

results = grid_search_l1.cv_results_
plt.figure(figsize=(5, 3))
plt.plot(param_grid["C"], results["mean_test_score"], marker="o")
plt.xscale("log")
plt.xlabel("Regularization strength (C)")
plt.ylabel("Mean CV Accuracy")
plt.title("Cross-Validation accuracy for different regularization strengths")
plt.grid(True)
plt.show()

In [ ]:
# Number of non-zero coefficients
(best_logreg_l1.coef_ != 0).sum()

In [ ]:
adata_hv.var["coef"] = best_logreg_l1.coef_[0]

mask = adata_hv.var["coef"].abs() > 0.7
plt.scatter(
    adata_hv.var["highly_variable_rank"],
    adata_hv.var["coef"],
)
for idx, row in adata_hv[:, mask].var.iterrows():
    plt.annotate(
        idx,
        (row["highly_variable_rank"], row["coef"]),
        textcoords="offset points",
        xytext=(0,10),
        ha="center",
    )
plt.xlabel("Gene rank")
plt.ylabel("Standardized coefficients")
plt.show()

In [ ]:
outfile = Path(datadir, "grid_search_l1_full_model.pkl")
if outfile.exists():
    with open(outfile, "rb") as f:
        full_models = pickle.load(f)
else:
    # Extract non-zero parameters for each search
    full_models = {}
    for i, params in enumerate(grid_search_l1.cv_results_["params"]):
        model = copy.deepcopy(grid_search_l1.estimator)
        full_models[i] = model.set_params(**params).fit(X_scaled_train, y_train)
    with open(outfile, "wb") as f:
        pickle.dump(full_models, f)

for i, params in enumerate(grid_search_l1.cv_results_["params"]):
    non_zero_params = (full_models[i].coef_ != 0).sum()
    print(f"Search {i + 1} with params {params}: {non_zero_params} non-zero parameters")

**NOTE:** Lasso regression can produce non-unique solutions, especially in cases where the data exhibits multicollinearity or when the number of features exceeds the number of observations. This non-uniqueness means that different runs can yield different sets of non-zero coefficients. Consequently, it is not surprising that the number of non-zero parameters differs between `best_logreg_l1` and the results above, as Lasso's selection of features can vary depending on the specific conditions of each run

## Elastic net

In [ ]:
# Define the logistic regression model
logreg = LogisticRegression(solver="saga", penalty="elasticnet", max_iter=3500)

# Define the parameter grid for C and l1_ratio
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "l1_ratio": [0.0, 0.25, 0.5, 0.75, 1.0],
}

# Perform cross-validation to find the best parameters
outfile = Path(datadir, "grid_search_elasticnet.pkl")
if outfile.exists():
    with open(outfile, "rb") as f:
        grid_search = pickle.load(f)
else:
    grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring="accuracy", verbose=1, n_jobs=-1)
    grid_search.fit(X_scaled_train, y_train)
    with open(outfile, "wb") as f:
        pickle.dump(grid_search, f)


# Get the best model and the best parameters
best_logreg = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")

# Train the logistic regression model with the best parameters on the entire training set
best_logreg.fit(X_scaled_train, y_train)

# Make predictions on the test set
y_pred = best_logreg.predict(X_scaled_test)

# Evaluate the model
print(f"Accuracy of training set: {best_logreg.score(X_scaled_train, y_train):.4f}")
print(f"Accuracy on test set: {best_logreg.score(X_scaled_test, y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Number of non-zero coefficients
(best_logreg.coef_ != 0).sum()

In [ ]:
adata_hv.var["coef"] = best_logreg.coef_[0]

mask = adata_hv.var["coef"].abs() > 0.7
plt.scatter(
    adata_hv.var["highly_variable_rank"],
    adata_hv.var["coef"],
)
for idx, row in adata_hv[:, mask].var.iterrows():
    plt.annotate(
        idx,
        (row["highly_variable_rank"], row["coef"]),
        textcoords="offset points",
        xytext=(0,10),
        ha="center",
    )
plt.xlabel("Gene rank")
plt.ylabel("Standardized coefficients")
plt.show()

The best solution was found with `l1_ratio=1`, which is equivalent to using an L1 penalty

# Bonus: Neural networks

Let's explore if using neural networks helps improving our classification task. Feel free to explore with the number of hidden layers and sizes, as well as the regularization strength (`alpha`)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver="adam", alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42, max_iter=500)
clf.fit(X_scaled_train, y_train)
clf.score(X_scaled_test, y_test)

In this case, using a more sophisticated model, such as a neural network, did not improve our prediction score compared to a simple logistic regression model. This outcome illustrates the principle that it is often better to start with simple models and only increase complexity when necessary. Simple models are easier to interpret, require less computational power, and are less prone to overfitting, especially when the dataset is small or not highly complex. Increasing model complexity should be considered when there is clear evidence that a more complex model can capture underlying patterns that a simpler model cannot